# LLM Queries
Bill Xia<br>
April 18, 2025

**Purpose:** A space to learn how to perform Neo4j queries using LangChain.

In [1]:
# Imports.
from json import load
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain.llms import Ollama
from langchain_core.prompts.prompt import PromptTemplate

In [2]:
# Load graph.
with open('../../../Documents/API_Keys/Neo4j-Instance01.json') as fp:
    credentials = load(fp)
assert credentials['NEO4J_URI'].startswith("neo4j+ssc://")

graph = Neo4jGraph(
    url      = credentials['NEO4J_URI'],
    username = credentials['NEO4J_USERNAME'],
    password = credentials['NEO4J_PASSWORD']
)
print(graph.schema)

Node properties:
User {username: STRING, is_private: BOOLEAN}
Pin {pin_id: INTEGER, caption: STRING, url: STRING}
Board {is_private: BOOLEAN, board_id: INTEGER, board_name: STRING}
ShareEvent {share_id: INTEGER, share_time: DATE_TIME}
Group {group_id: INTEGER, permissions: STRING}
Relationship properties:

The relationships:
(:User)-[:CREATED]->(:Pin)
(:User)-[:CREATED]->(:Board)
(:User)-[:CREATED]->(:Group)
(:User)-[:FOLLOWS]->(:User)
(:User)-[:FOLLOWS]->(:Board)
(:User)-[:BLOCKS]->(:User)
(:User)-[:SHARES]->(:ShareEvent)
(:User)-[:SAVES]->(:Pin)
(:Board)-[:CONTAINS]->(:Pin)
(:ShareEvent)-[:DELIVERS]->(:Pin)
(:ShareEvent)-[:HAS_RECIPIENT]->(:User)
(:Group)-[:HAS_MEMBER]->(:User)
(:Group)-[:CAN_ACCESS]->(:Board)


In [3]:
# Load LLM.
model_name = 'llama3.2:latest'
llm = Ollama(model=model_name)

/var/folders/wj/59p675nx1ylcm5hhmc_djrp80000gn/T/ipykernel_82080/3340281701.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=model_name)


In [4]:
# Initialize QA chain.
chain = GraphCypherQAChain.from_llm(
    llm, graph=graph, verbose=True, allow_dangerous_requests=True
)

In [5]:
# Running the chain.
query = "How many followers does Cindy have?"
response = chain.run(query)
print(response)



> Entering new GraphCypherQAChain chain...


/var/folders/wj/59p675nx1ylcm5hhmc_djrp80000gn/T/ipykernel_82080/3921008535.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(query)


Generated Cypher:
MATCH (u:User {username: 'Cindy'})-[:FOLLOWS]-(f:User) RETURN COUNT(f)
Full Context:
[{'COUNT(f)': 3}]

> Finished chain.
I don't know the answer.


## Checkpoint
We've figured out how to query the database using an LLM as an intermediary.
Now we need to tune our prompt so that the LLM (which in this case, is pretty
weak) can understand what gets returned by the graph query.

In [6]:
prompt_template = """Task: Generate Cypher code to query to a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question}
"""
full_prompt = PromptTemplate(
    input_variables = ['schema', 'question'],
    template        = prompt_template
)

In [ ]:
# Load the chain.
chain = GraphCypherQAChain.from_llm(
    llm,
    graph                    = graph,
    verbose                  = True,
    cypher_prompt            = full_prompt,
    allow_dangerous_requests = True
)

In [8]:
# Running the chain.
query    = "Who created pin 0?"
response = chain.invoke({"query": query})
print(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Pin {pin_id: 0})-[:CREATED]->(u:User) RETURN u.username
Full Context:
[]

> Finished chain.
{'query': 'Who created pin 0?', 'result': "I don't have enough information to provide an answer."}
